Aim: classification on mbay songs: given a mbay song, can we predict its genre?

In [0]:
# Help: https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-1.html

# Files: https://drive.google.com/drive/folders/0B-znYQjmJbqKTWVGUG9mQVNPZTQ

In [0]:
# loading packages

%tensorflow_version 1.x
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
import keras
from keras.models import Sequential
import warnings
from sklearn.metrics import classification_report,confusion_matrix
from numpy import argmax
import librosa.display
import IPython.display
import random
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.optimizers import SGD

warnings.filterwarnings('ignore')

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
# Work directory

from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
duree = 30 # sample duration to take from the audio file for analysis
genres = 'Bang Bordero Klag Baïgn'.split()
MaxNbFiles = 75 # maximum number of songs of a genre to consider for analysis

Waveplots and spectograms of a sample of each genre

In [0]:
pathlib.Path(f'img_data/').mkdir(parents=True, exist_ok=True)

cmap = plt.get_cmap('inferno')
plt.figure(figsize=(14,5))

for g in genres:
  filename = os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}')[0]
  songname = f'/content/gdrive/My Drive/Discothèque Mbaye/{g}/{filename}'

  x, srx = librosa.load(songname, mono=True)
  y, sr = librosa.load(songname, mono=True,offset=librosa.get_duration(x)/2, duration=duree)

  librosa.display.waveplot(y, sr=sr,offset=librosa.get_duration(x)/2);
  plt.savefig(f'img_data/wave_{g}.png')
  plt.clf()

  plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
  plt.savefig(f'img_data/spect_{g}.png')
  plt.clf()

<Figure size 1008x360 with 0 Axes>

Working on features from spectogram (stored in a csv file)

In [0]:
# # Read the songs, extract the features and write the data in a csv file

# Header for our CSV file

header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

# Extracting features from Spectrogram: Mel-frequency cepstral coefficients (MFCC), Spectral Centroid, Zero Crossing Rate, Chroma Frequencies, and Spectral Roll-off

file = open('dataset_'+str(duree)+'s.csv', 'w', newline='') # 'a' for openning an existing file for writing
with file:
    writer = csv.writer(file,delimiter=';')
    writer.writerow(header)

for g in genres:
  NbFiles = len(os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}'))
  i = 1
  while (i <= min(MaxNbFiles,NbFiles)):
    filename = os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}')[i-1]
    songname = f'/content/gdrive/My Drive/Discothèque Mbaye/{g}/{filename}'
    x, srx = librosa.load(songname, mono=True)
    y, sr = librosa.load(songname, mono=True,offset=librosa.get_duration(x)/2, duration=duree)
    rmse = librosa.feature.rmse(y=y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    filename = 'song' + str(i)
    to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
      to_append += f' {np.mean(e)}'
    to_append += f' {g}'
    file = open('dataset_'+str(duree)+'s.csv', 'a', newline='')
    with file:
      writer = csv.writer(file,delimiter=';')
      writer.writerow(to_append.split())
    i=i+1

In [0]:
# Reading the csv file

data = pd.read_csv('/content/dataset_30s.csv', sep=';')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

# Converting to numeric type
numeric = list(data.columns)
del numeric[-1]
data[numeric] = data[numeric].apply(pd.to_numeric, errors='coerce')

#data['label'].unique() # checking we have all the genres

# Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

# Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0,test_size=0.3)

print(X_train.shape); print(X_test.shape)

(123, 26)
(53, 26)


Multi-Layer Perceptron Classifier

In [0]:
# From https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
y_pred = mlp.fit(X_train,y_train).predict(X_test)
#print(confusion_matrix(y_test,predict_test))
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.78      0.69      0.73        26
           1       0.00      0.00      0.00         0
           2       0.81      0.74      0.77        23
           3       0.38      0.75      0.50         4

    accuracy                           0.72        53
   macro avg       0.49      0.55      0.50        53
weighted avg       0.76      0.72      0.73        53



Artificial Neural Networks

In [0]:
ANN = Sequential();
ANN.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)));
ANN.add(layers.Dense(128, activation='relu'));
ANN.add(layers.Dense(64, activation='relu'));
ANN.add(layers.Dense(10, activation='softmax'));
ANN.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']);

ANN.fit(X_train,y_train,epochs=100,batch_size=128);

y_pred = ANN.predict_classes(X_test)
print(classification_report(y_pred,y_test))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/100
123/123 [==============================] - 0s 1ms/step - loss: 2.4872 - accuracy: 0.0000e+00
Epoch 2/100
123/123 [==============================] - 0s 55us/step - loss: 2.3567 - accuracy: 0.0894
Epoch 3/100
123/123 [==============================] - 0s 53us/step - loss: 2.2048 - accuracy: 0.3171
Epoch 4/100
123/123 [==============================] - 0s 48us/step - loss: 2.0868 - accuracy: 0.4228
Epoch 5/100
123/123 [==============================] - 0s 39us/step - loss: 1.9699 - accuracy: 0.4878
Epoch 6/100
123/123 [==============================] - 0s 44us/step - loss: 1.8620 - accuracy: 0.5203
Epoch 7/100
123/123 [==============================] - 0s 75us/step - loss: 1.7550 - accuracy: 0.5366
Epoch 8/100
123/123 [==============================] - 0s 57us/step - loss: 1.6499 - accuracy: 0.5447
Epoch 9/100
123/123 [==============================] - 0s 56us/step - loss: 1.5469 - accuracy: 0.57

Multinomial regression

In [0]:
from sklearn.linear_model import LogisticRegression

multi = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = multi.predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.65      0.68      0.67        22
           1       0.00      0.00      0.00         1
           2       0.86      0.69      0.77        26
           3       0.38      0.75      0.50         4

    accuracy                           0.68        53
   macro avg       0.47      0.53      0.48        53
weighted avg       0.72      0.68      0.69        53



K-Nearest Neighbors

In [0]:
# https://scikit-learn.org/stable/modules/neighbors.html#classification
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py

from sklearn import neighbors, datasets

n_neighbors = 15
nghbs = neighbors.KNeighborsClassifier(n_neighbors).fit(X_train, y_train)
y_pred = nghbs.predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.65      0.71      0.68        21
           1       0.00      0.00      0.00         0
           2       0.95      0.67      0.78        30
           3       0.25      1.00      0.40         2

    accuracy                           0.70        53
   macro avg       0.46      0.60      0.47        53
weighted avg       0.81      0.70      0.73        53



Support Vector Machines

In [0]:
from sklearn import svm

SVM_model = svm.SVC()#(kernel='linear')
SVM_model.fit(X_train,y_train)
y_pred = SVM_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.74      0.74        23
           1       0.00      0.00      0.00         1
           2       0.77      0.95      0.85        21
           3       0.67      0.25      0.36         8

    accuracy                           0.74        53
   macro avg       0.54      0.49      0.49        53
weighted avg       0.73      0.74      0.71        53



Random Forests

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

ForAlea = RandomForestClassifier(max_depth=5, random_state=0)
y_pred = ForAlea.fit(X_train, y_train).predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.72      0.78      0.75        23
           1       0.00      0.00      0.00         1
           2       0.70      0.90      0.79        21
           3       1.00      0.12      0.22         8

    accuracy                           0.72        53
   macro avg       0.61      0.45      0.44        53
weighted avg       0.74      0.72      0.67        53



Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.48      0.79      0.59        14
           1       0.00      0.00      0.00         5
           2       0.86      0.64      0.73        28
           3       0.38      0.50      0.43         6

    accuracy                           0.60        53
   macro avg       0.43      0.48      0.44        53
weighted avg       0.62      0.60      0.59        53



Linear Discriminant Analysis

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LinDis = LinearDiscriminantAnalysis()
y_pred = LinDis.fit(X_train, y_train).predict(X_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.74      0.74      0.74        23
           1       0.00      0.00      0.00         3
           2       0.81      0.77      0.79        22
           3       0.38      0.60      0.46         5

    accuracy                           0.70        53
   macro avg       0.48      0.53      0.50        53
weighted avg       0.69      0.70      0.69        53



Convolutional Neural Networks

In [0]:
# https://levelup.gitconnected.com/audio-data-analysis-using-deep-learning-with-python-part-2-4a1f40d3708d
# https://missinglink.ai/guides/convolutional-neural-networks/python-convolutional-neural-network-creating-cnn-keras-tensorflow-plain-python/

# Extracting spectograms

cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))

for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    NbFiles = len(os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}'))
    i=1
    while (i <= min(MaxNbFiles,NbFiles)):
        filename = os.listdir(f'/content/gdrive/My Drive/Discothèque Mbaye/{g}')[i-1]
        songname = f'/content/gdrive/My Drive/Discothèque Mbaye/{g}/{filename}'
        x, srx = librosa.load(songname, mono=True)
        y, sr = librosa.load(songname, mono=True,offset=librosa.get_duration(x)/2, duration=duree)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        filename = 'chanson' + str(i)
        plt.savefig(f'img_data/{g}/{filename}.png')
        plt.clf()
        i=i+1

In [0]:
# Split the data into the train set and test set
!pip install split_folders

import split_folders

split_folders.ratio('/content/img_data', output="/content/data", seed=0, ratio=(.7, .3))

Copying files: 176 files [00:00, 1225.25 files/s]


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
        #rescale=1./255, # rescale all pixel values from 0-255, so after this step all our pixel values are in range (0,1)
        #shear_range=0.2, #to apply some random transformations
        #zoom_range=0.2, #to apply zoom
        #horizontal_flip=True) # image will be flipped horizontally

test_datagen = ImageDataGenerator(rescale=1./255) #rescale=1./255

training_set = train_datagen.flow_from_directory(
        '/content/data/train',
        target_size=(64, 64),
        batch_size=25,
        classes = genres,
        class_mode='categorical',
        shuffle = False)

test_set = test_datagen.flow_from_directory(
        '/content/data/val',
        target_size=(64, 64),
        batch_size=25,
        classes = genres,
        class_mode='categorical',
        shuffle = False )

Found 129 images belonging to 4 classes.
Found 63 images belonging to 4 classes.


In [0]:
model = Sequential()
input_shape=(64, 64, 3)

#1st hidden layer
model.add(Conv2D(64, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))

#Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

#Output layer
model.add(Dense(4))
model.add(Activation('softmax'))
#model.summary()

epochs = 200
batch_size = 25
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

model.fit_generator(training_set,
        #steps_per_epoch=5,
        epochs=100 #,validation_data=test_set,validation_steps=25
        )


Epoch 1/100
6/6 [==============================] - 2s 337ms/step - loss: 1.4604 - accuracy: 0.1705
Epoch 2/100
6/6 [==============================] - 2s 333ms/step - loss: 1.3914 - accuracy: 0.2946
Epoch 3/100
6/6 [==============================] - 2s 341ms/step - loss: 1.3750 - accuracy: 0.3876
Epoch 4/100
6/6 [==============================] - 2s 339ms/step - loss: 1.3780 - accuracy: 0.4109
Epoch 5/100
6/6 [==============================] - 2s 339ms/step - loss: 1.3845 - accuracy: 0.2946
Epoch 6/100
6/6 [==============================] - 2s 340ms/step - loss: 1.4037 - accuracy: 0.3256
Epoch 7/100
6/6 [==============================] - 2s 348ms/step - loss: 1.3684 - accuracy: 0.4264
Epoch 8/100
6/6 [==============================] - 2s 343ms/step - loss: 1.3656 - accuracy: 0.4109
Epoch 9/100
6/6 [==============================] - 2s 351ms/step - loss: 1.3431 - accuracy: 0.4031
Epoch 10/100
6/6 [==============================] - 2s 343ms/step - loss: 1.3533 - accuracy: 0.4264
Epoch 11

In [0]:
test_set.reset()
y_pred = model.predict_generator(test_set)

predicted_class_indices=np.argmax(y_pred,axis=1)
print(classification_report(test_set.classes,predicted_class_indices))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.43      1.00      0.60        27
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00         4

    accuracy                           0.43        63
   macro avg       0.11      0.25      0.15        63
weighted avg       0.18      0.43      0.26        63

